In [74]:
# Import Depedencies 
import pandas as pd 
import matplotlib as plt
import requests
import gmaps

# Getting API key
from configs import gkey

In [64]:
# Import cleaned csv
csvfile1 = "Resources/Cleaned_Vaccine_Case_Data.csv"

# Add as DataFrame
main_df = pd.read_csv(csvfile1)
main_df = main_df.drop(columns="Unnamed: 0")
main_df

,POSTCODE,Total Population,Total Case Count,% Cases Over Age 18,% Cases Over Age 65,Total Cases Category,Total Vaccinated,Total Vaccinated Category,Median Household Income
0,85003,8499,1534,46.20,74.49,Greater Than 1000,4898,"1,001-5,000",44140
1,85004,7164,969,63.68,61.45,501-1000,4281,"1,001-5,000",41951
2,85006,27206,3272,35.85,54.55,Greater Than 1000,6780,"5,001-10,000",38121
3,85007,14428,2424,41.44,63.91,Greater Than 1000,4719,"1,001-5,000",34667
4,85008,62340,9079,33.01,56.27,Greater Than 1000,14187,"Greater Than 10,000",43574
...,...,...,...,...,...,...,...,...,...
339,86444,1254,60,34.27,49.25,11-100,450,"11-1,000",46389
340,86445,254,12,12.73,20.45,11-100,42,"11-1,000",51150
341,86502,1095,497,34.12,47.10,101-500,265,"11-1,000",18542
342,86505,8036,137,12.40,36.62,101-500,54,"11-1,000",23767


In [98]:
# Minor cleaning of dataframe
main_df = main_df.loc[main_df["Total Case Count"] != "1-10", :]
main_df = main_df.loc[main_df["Total Vaccinated"] != "1-10", :]
main_df = main_df.loc[main_df["Median Household Income"] != "-", :]
main_df = main_df.loc[main_df["Total Population"] != "1-10", :]
main_df = main_df.loc[main_df["Total Population"] != "0", :]
# Creating new columns to hold general lat-lng of the zipcodes
main_df["Lat"] = " "
main_df["Lng"] = " "
# Converting numeric values back to int datatype
main_df['Total Population'] = main_df['Total Population'].astype(int)
main_df['Total Case Count'] = main_df['Total Case Count'].astype(int)
main_df['Total Vaccinated'] = main_df['Total Vaccinated'].astype(int)
main_df['Median Household Income'] = main_df['Median Household Income'].astype(int)
# Show main_df
main_df

,level_0,index,POSTCODE,Total Population,Total Case Count,% Cases Over Age 18,% Cases Over Age 65,Total Cases Category,Total Vaccinated,Total Vaccinated Category,Median Household Income,Lat,Lng
0,0,0,85003,8499,1534,46.20,74.49,Greater Than 1000,4898,"1,001-5,000",44140,,
1,1,1,85004,7164,969,63.68,61.45,501-1000,4281,"1,001-5,000",41951,,
2,2,2,85006,27206,3272,35.85,54.55,Greater Than 1000,6780,"5,001-10,000",38121,,
3,3,3,85007,14428,2424,41.44,63.91,Greater Than 1000,4719,"1,001-5,000",34667,,
4,4,4,85008,62340,9079,33.01,56.27,Greater Than 1000,14187,"Greater Than 10,000",43574,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,315,337,86442,33895,4071,27.17,48.51,Greater Than 1000,7922,"5,001-10,000",39600,,
316,316,339,86444,1254,60,34.27,49.25,11-100,450,"11-1,000",46389,,
317,317,340,86445,254,12,12.73,20.45,11-100,42,"11-1,000",51150,,
318,318,341,86502,1095,497,34.12,47.10,101-500,265,"11-1,000",18542,,


In [73]:
# Query URL
query_url = "https://maps.googleapis.com/maps/api/geocode/json?"

# FOR loop for looping and adding lat-lng for the zipcodes
for index, zips in main_df.iterrows():
    params = {
        "key": gkey,
        "address": zips["POSTCODE"],
    }
    response = requests.get(query_url, params = params).json()
    
    try:
        main_df.loc[index, "Lat"] = response["results"][0]["geometry"]["location"]["lat"]
        main_df.loc[index, "Lng"] = response["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print(f"An error occurred with this zipcode. Skipping...")

main_df.head()   

,POSTCODE,Total Population,Total Case Count,% Cases Over Age 18,% Cases Over Age 65,Total Cases Category,Total Vaccinated,Total Vaccinated Category,Median Household Income,Lat,Lng
0,85003,8499,1534,46.20,74.49,Greater Than 1000,4898,"1,001-5,000",44140,33.4496,-112.079
1,85004,7164,969,63.68,61.45,501-1000,4281,"1,001-5,000",41951,33.4528,-112.069
2,85006,27206,3272,35.85,54.55,Greater Than 1000,6780,"5,001-10,000",38121,33.4636,-112.054
3,85007,14428,2424,41.44,63.91,Greater Than 1000,4719,"1,001-5,000",34667,33.4463,-112.089
4,85008,62340,9079,33.01,56.27,Greater Than 1000,14187,"Greater Than 10,000",43574,33.466,-112.012


In [105]:
# Setting up basemap and adding heatmap layer for Case Count vs Population in Zipcodes
# Grabbing Locations
locations = [(main_df.iloc[i, -2], main_df.iloc[i, -1]) for i in main_df.index]
# Setting weights (Case Count vs Population)
weights = [(main_df.iloc[i, 2] / main_df.iloc[i, 1] * 100) for i in main_df.index]
# Configure GMaps
'''
gmaps.configure(api_key=gkey)
fig = gmaps.figure(map_type='ROADMAP', zoom_level=5, center=(34.0489, 111.0937))
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap_layer)
fig
'''
for i in range(len(main_df.index)):
    if main_df.iloc[i, 1] == 0:
        print(main_df.iloc[i, 0])

0
ipykernel_launcher:5: RuntimeWarning: divide by zero encountered in longlong_scalars
